<a href="https://colab.research.google.com/github/Hussain1488/Fake-News-Detection/blob/main/FakeNewsDetection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# !pip install torchtext kagglehub
# !pip uninstall -y torch torchtext
!pip install torch==2.3.0 torchtext==0.18.0


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.2/779.2 MB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 54.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 95.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 78.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.2/176.2 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9

In [ ]:
import kagglehub
import torchtext
import pandas as pd
import numpy as np
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from torchtext.data.utils import get_tokenizer
from nltk.stem import PorterStemmer
from nltk.stem import PorterStemmer




[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
path = kagglehub.dataset_download("clmentbisaillon/fake-and-real-news-dataset")
print("Path to dataset files:", path)

stop_words = set(stopwords.words('english'))
print(stop_words)

Path to dataset files: /kaggle/input/fake-and-real-news-dataset
{"wouldn't", "they've", 'yourselves', 'because', "won't", 'haven', 'both', "you'd", 'same', 'any', 'such', 'yours', 'now', 'will', 'all', 'there', 'hers', 'this', "mustn't", 'd', 'be', 'itself', 'himself', 'its', "it'll", 'me', 'through', "we're", 'my', 'ourselves', 'were', 'ours', "you're", 'once', 'if', 'isn', "weren't", 'and', "that'll", 'being', 'yourself', 'why', 'mustn', 'has', 'don', 'needn', 'aren', 'whom', 'they', 'wasn', 'should', 'your', "shan't", 'how', 'very', 'shouldn', 'against', 'you', 'over', 'mightn', 'he', 'on', 'down', "isn't", "don't", 'in', 'an', "mightn't", "he'd", 'ma', 'are', 'for', 'of', "didn't", 'between', 'myself', 'theirs', 'which', 'them', 'doesn', 'his', 'a', 'after', 'it', "i've", 'own', "i'm", 'off', 'her', 'ain', 'couldn', 'i', 'had', 'until', "hasn't", 's', 'other', "she'll", 't', 'then', 'when', 'under', 'have', 'does', 'or', 'too', "she'd", 'here', 'themselves', 'each', "doesn't", 're'

# Data Visualization

In [ ]:

fake=pd.read_csv(path + '/Fake.csv')
real=pd.read_csv(path + '/True.csv')

for i, item in fake.iterrows():
    filtered_text = [token.lower() for token in item['text'].split() if token.lower() not in stop_words]
    filtered_title = [token.lower() for token in item['title'].split() if token.lower() not in stop_words]
    fake.at[i, 'text'] = filtered_text
    fake.at[i, 'title'] = filtered_title

for i, item in real.iterrows():
    filtered_text = [token.lower() for token in item['text'].split() if token.lower() not in stop_words]
    filtered_title = [token.lower() for token in item['title'].split() if token.lower() not in stop_words]
    real.at[i, 'text'] = filtered_text
    real.at[i, 'title'] = filtered_title

print(fake.head())
print(real.head())
fake['label'] = 0
real['label'] = 1
df = pd.concat([fake, real])
df

                                               title  \
0  [donald, trump, sends, embarrassing, new, year...   
1  [drunk, bragging, trump, staffer, started, rus...   
2  [sheriff, david, clarke, becomes, internet, jo...   
3  [trump, obsessed, even, obama’s, name, coded, ...   
4  [pope, francis, called, donald, trump, christm...   

                                                text subject  \
0  [donald, trump, wish, americans, happy, new, y...    News   
1  [house, intelligence, committee, chairman, dev...    News   
2  [friday,, revealed, former, milwaukee, sheriff...    News   
3  [christmas, day,, donald, trump, announced, wo...    News   
4  [pope, francis, used, annual, christmas, day, ...    News   

                date  
0  December 31, 2017  
1  December 31, 2017  
2  December 30, 2017  
3  December 29, 2017  
4  December 25, 2017  
                                               title  \
0  [u.s., budget, fight, looms,, republicans, fli...   
1  [u.s., military, accept, 

,title,text,subject,date,label
0,"[donald, trump, sends, embarrassing, new, year...","[donald, trump, wish, americans, happy, new, y...",News,"December 31, 2017",0
1,"[drunk, bragging, trump, staffer, started, rus...","[house, intelligence, committee, chairman, dev...",News,"December 31, 2017",0
2,"[sheriff, david, clarke, becomes, internet, jo...","[friday,, revealed, former, milwaukee, sheriff...",News,"December 30, 2017",0
3,"[trump, obsessed, even, obama’s, name, coded, ...","[christmas, day,, donald, trump, announced, wo...",News,"December 29, 2017",0
4,"[pope, francis, called, donald, trump, christm...","[pope, francis, used, annual, christmas, day, ...",News,"December 25, 2017",0
...,...,...,...,...,...
21412,"['fully, committed', nato, backs, new, u.s., a...","[brussels, (reuters), -, nato, allies, tuesday...",worldnews,"August 22, 2017",1
21413,"[lexisnexis, withdrew, two, products, chinese,...","[london, (reuters), -, lexisnexis,, provider, ...",worldnews,"August 22, 2017",1
21414,"[minsk, cultural, hub, becomes, authorities]","[minsk, (reuters), -, shadow, disused, soviet-...",worldnews,"August 22, 2017",1
21415,"[vatican, upbeat, possibility, pope, francis, ...","[moscow, (reuters), -, vatican, secretary, sta...",worldnews,"August 22, 2017",1


In [ ]:
df.isnull().sum()


,0
title,0
text,0
subject,0
date,0
label,0


In [ ]:

# Make sure columns can hold lists
df['text'] = df['text'].astype(object)
df['title'] = df['title'].astype(object)


stemmer = PorterStemmer()

# Stem text column
df['text'] = df['text'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])

# Stem title column
df['title'] = df['title'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])

# Check first row
print(df['text'][0])
print(df['title'][0])



0    [donald, trump, wish, american, happi, new, ye...
0    [washington, (reuters), -, head, conserv, repu...
Name: text, dtype: object
0    [donald, trump, send, embarrass, new, year’, e...
0    [u.s., budget, fight, looms,, republican, flip...
Name: title, dtype: object
